In [486]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [487]:
# Features
# X = ['GDPPerCapita', 'CostOfSolar', 'CostOfBatt', 'Population]
# Y = Solar MW Installed

In [488]:
techcostvar = "Solar_Generation_Cost_USD_MW"
battscenario = "Mid_Bat_Cost_USD_MW"

In [489]:
years = ["2000","2012","2013","2014","2015","2016","2017","2018","2019"]

Popfp = '.\Data\PopAndGDP\Population.csv'
Pop = pd.read_csv(Popfp, encoding = "utf-8")
Pop = Pop.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="population")
Pop["Year"] = Pop["Year"].apply(pd.to_numeric, errors='coerce')

GDPfp = '.\Data\PopAndGDP\GDPPerCapita.csv'
GDP = pd.read_csv(GDPfp, encoding = "utf-8")
GDP = GDP.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="GDP_Per_Capita")
GDP["Year"] = GDP["Year"].apply(pd.to_numeric, errors='coerce')

TechCostfp = '.\Data\SolarWindConstructionCost\SolarWindConstructionCost.csv'
TechCost = pd.read_csv(TechCostfp, encoding="utf-8")
TechCost = TechCost.apply(pd.to_numeric, errors='coerce')

SolarCost = TechCost[["Year", techcostvar]]
BattCost = TechCost[["Year", battscenario]]

Installedfp = '.\Data\InstalledCapacity\RECAP_20220519-053554.csv'
Installed = pd.read_csv(Installedfp, encoding = "ISO-8859-1")
Installed = Installed[Installed.Technology == "Solar"]
Installed = Installed.melt(id_vars=["Country", "Technology"], var_name="Year", value_name="Installed_Solar_Capacity")
Installed["Year"] = Installed["Year"].apply(pd.to_numeric, errors='coerce')

In [490]:
df = pd.merge(Pop, GDP)
df = df.merge(SolarCost, on="Year")
df = df.merge(BattCost, on="Year")
df = df.merge(Installed, on=["Country", "Year"])
df.head()


,Country,Country Code,Year,population,GDP_Per_Capita,Solar_Generation_Cost_USD_MW,Mid_Bat_Cost_USD_MW,Technology,Installed_Solar_Capacity
0,Afghanistan,AFG,2012,31161378,638.8458516,335.66,640,Solar,16.122
1,Albania,ALB,2012,2900401,4247.630047,335.66,640,Solar,0.679
2,Algeria,DZA,2012,37383899,5592.220115,335.66,640,Solar,25
3,Andorra,AND,2012,82427,38686.46126,335.66,640,Solar,0
4,Angola,AGO,2012,25107925,5100.097027,335.66,640,Solar,9.604


In [491]:
X_params = ["population", "GDP_Per_Capita",techcostvar,battscenario]
Y_param = ["Installed_Solar_Capacity"]

df.replace('..', np.nan, inplace=True)
df = df[X_params + Y_param].dropna()
#df.to_csv("traindataset.csv")


In [492]:

X = df[X_params]
X = X.apply(pd.to_numeric, errors='coerce')
Y = df[Y_param]
Y = Y.apply(pd.to_numeric, errors='coerce')


In [493]:
X.to_csv("Xtraindataset.csv")
Y.to_csv("Ytraindataset.csv")


https://www.weforum.org/agenda/2021/11/renewable-energy-cost-fallen/#:~:text=Back%20in%202010%2C%20a%20megawatt,been%20applicable%20in%20some%20areas.
https://www.windpowermonthly.com/article/1713420/windeconomics-fall-fall-wind-solar-costs
https://www.nrel.gov/docs/fy21osti/79236.pdf

In [494]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

In [495]:
print("X_train shape:" , X_train.shape, "X_test shape:", X_test.shape)
print("y_train shape:" , y_train.shape, "y_test shape:", y_test.shape)

X_train shape: (927, 4) X_test shape: (457, 4)
y_train shape: (927, 1) y_test shape: (457, 1)


In [496]:
type(y_train)

pandas.core.frame.DataFrame

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [501]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(4, activation='relu',input_dim=4))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dense(1))


In [502]:
model.compile(
    optimizer='adam', 
    loss="mean_squared_error",
    metrics=['accuracy'])

In [503]:
print(model.summary())

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 4)                 20        
                                                                 
 dense_79 (Dense)            (None, 4)                 20        
                                                                 
 dense_80 (Dense)            (None, 4)                 20        
                                                                 
 dense_81 (Dense)            (None, 1)                 5         
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________
None


In [504]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
29/29 [==============================] - 0s 2ms/step - loss: 1338481152.0000 - accuracy: 0.0302  
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 1338481152.0000 - accuracy: 0.0302
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 1338481024.0000 - accuracy: 0.0302
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 1338480640.0000 - accuracy: 0.0302
Epoch 5/100
29/29 [==============================] - 0s 1ms/step - loss: 1338480640.0000 - accuracy: 0.0302
Epoch 6/100
29/29 [==============================] - 0s 1ms/step - loss: 1338480128.0000 - accuracy: 0.0302
Epoch 7/100
29/29 [==============================] - 0s 1ms/step - loss: 1338480384.0000 - accuracy: 0.0302
Epoch 8/100
29/29 [==============================] - 0s 1ms/step - loss: 1338480128.0000 - accuracy: 0.0302
Epoch 9/100
29/29 [==============================] - 0s 1ms/step - loss: 1338480128.0000 - accuracy: 0.0302
Epoch 10/100
29/29 [======

In [505]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

15/15 - 0s - loss: 943944768.0000 - accuracy: 0.0022 - 109ms/epoch - 7ms/step

Test accuracy: 0.002188183832913637


In [506]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)

15/15 [==============================] - 0s 1ms/step


In [507]:
predictions

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],